## The idea 

The idea is to build an admin panel for apps using FastHTML and SQLite. I wanna use fastlite to talk to the database and fhdaisy to style the app. The idea is to have this a standalone app that I can mount into another app.

Please use your tools to change the messages directly when I prompt you to change something.

## Setup

In [ ]:
#| default_exp core

In [ ]:
#| export
from fasthtml.common import *
from fasthtml.jupyter import * 
from fhdaisy import *

from fastcore.utils import *

from fh_heroicons import Heroicon 

from fasthtml.common import Input as Input_

from pathlib import Path

from functools import cached_property

In [ ]:
from fastcore.net import urlsave

In [ ]:
#| export
app = FastHTML(hdrs=(daisy_hdrs,), session_cookie="fastadmin_")

rt = app.route

In [ ]:
p = mk_previewer(app)

## Database

For development we'll use the `chinook` sqlite database. This cell downloads it in the right place if you don't have it already downloaded.

In [ ]:
url = 'https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite'
path = Path('../data/chinook.sqlite')
if not path.exists(): urlsave(url, path)

In [ ]:
db = database("../data/chinook.sqlite")

## Configuration

In [ ]:
#| export
@dataclass
class AdminConfig:
    db_path: str | Path
    password: str = "admin123"
    
    @cached_property
    def db(self): return database(self.db_path)

In [ ]:
cfg = AdminConfig(db_path="../data/chinook.sqlite", password="admin123")

In [ ]:
#| export
def create_admin(config):
    app.state.cfg = config
    return app

In [ ]:
app = create_admin(cfg)

## Utility Functions

In [ ]:
#| export
def tbl_name(t): return t.name.strip('"') if hasattr(t, 'name') else str(t).strip('"')

In [ ]:
#| export
def mock_req(root_path=""):
    from starlette.requests import Request
    return Request({"type": "http", "root_path": root_path, "headers": []})

_mock_req = mock_req()

def url(req, path): return f"{req.scope.get('root_path', '')}{path}"

In [ ]:
#| export
def get_col_types(db, t):
    name = tbl_name(t)
    return {r[1]: r[2] for r in db.execute(f'PRAGMA table_info("{name}")').fetchall()}

In [ ]:
#| export
def get_cols(db, t):
    name = tbl_name(t)
    return [r[1] for r in db.execute(f'PRAGMA table_info("{name}")').fetchall()]

In [ ]:
#| export
def get_text_cols(db, t):
    return [name for name, typ in get_col_types(db, t).items() if 'TEXT' in typ.upper() or 'CHAR' in typ.upper()]

## UI Components

In [ ]:
#| export
def LabeledInput(icon_name, **kwargs):
    return Label(
        Span(Heroicon(icon_name), cls="label"),
        Input_(cls="w-full", **kwargs),
        cls="input w-full"
    )

In [ ]:
p(LabeledInput("lock-closed", placeholder="Password"))

In [ ]:
#| export
def HeaderBar(*,req=_mock_req, logged_in=False):
    logout_btn = Form(method="post", action=url(req, "/logout"))(
        Button(Heroicon("arrow-right-on-rectangle"), "Logout", cls="btn btn-ghost btn-sm gap-1")) if logged_in else None
    return Div(cls="navbar bg-base-200 mb-6")(
        Div(cls="flex-1")(A("fhadmin", href=url(req, "/tables"), cls="btn btn-ghost text-xl")),
        Div(cls="flex-none")(logout_btn))

In [ ]:
p(HeaderBar())

In [ ]:
#| export
def Layout(*children, title="Admin", req=_mock_req, logged_in=False):
    return (Title(title), 
            HeaderBar(req=req, logged_in=logged_in),
            Main(cls="max-w-[1800px] mx-auto px-6 py-8")(*children))

In [ ]:
#| export
def NewRowBtn(tbl, *, req=_mock_req): return Button(Heroicon("plus"), "New Record", cls="btn btn-primary btn-sm flex items-center hap-2", hx_get=url(req, f"/tables/{tbl}/new"), hx_target="body", hx_swap="beforeend")

In [ ]:
#| export
def EditableCell(tbl, pk, col, val, *, req=_mock_req):
    return Td(str(val), hx_get=url(req, f"/tables/{tbl}/{pk}/edit/{col}"), hx_swap="outerHTML", cls="cursor-pointer hover:bg-base-200")

In [ ]:
def EditCellInput(tbl, pk, col, val, *, req=_mock_req):
    return Td(Input(value=val, name=col, cls="-bordered -sm w-full", autofocus=True,
                     hx_put=url(req, f"/tables/{tbl}/{pk}/edit/{col}"), hx_target="closest td", hx_trigger="blur, keyup[key=='Enter']", hx_swap="outerHTML"))

In [ ]:
#| export
def TableRows(rows, cols, tbl, *, req=_mock_req):
    pk_col = cols[0]
    return [Tr(*[EditableCell(tbl, r[pk_col], c, r.get(c, ''), req=req) for c in cols],
               Td(Button(Heroicon("trash"), cls="btn btn-ghost btn-xs", hx_delete=url(req, f"/tables/{tbl}/{r[pk_col]}"), hx_target="closest tr", hx_swap="outerHTML")))
            for r in rows]

In [ ]:
#| export
def NewRowModal(tbl, cols, *, req=_mock_req):
    inputs = [Div(cls="form-control")(Label(c, cls="label"), Input_(name=c, cls="input input-bordered w-full")) for c in cols[1:]]
    return Dialog(id="new-row-modal", cls="modal modal-open")(
        Div(cls="modal-box")(
            H3(f"New {tbl} record", cls="font-bold text-lg mb-4"),
            Form(method="post", action=url(req, f"/tables/{tbl}/new"), hx_post=url(req, f"/tables/{tbl}/new"), hx_target="#table-content", hx_swap="outerHTML")(*inputs,
                Div(cls="modal-action")(
                    Button("Cancel", cls="btn", type="button", onclick="document.getElementById('new-row-modal').remove()"),
                    Button("Create", cls="btn btn-primary", type="submit")))))

In [ ]:
#| export
def SqlConsole(tbl, *, req=_mock_req):
    return Div(
        H3("SQL Console", cls="font-bold mb-2"),
        Form(hx_post=url(req, f"/tables/{tbl}/sql"), hx_target="#sql-results", hx_swap="innerHTML", cls="flex flex-col gap-2")(
            Textarea(name="sql", placeholder=f"SELECT * FROM {tbl} LIMIT 10", cls="textarea textarea-bordered w-full font-mono", rows=3),
            Div(cls="flex gap-2")(
                Button("Run", cls="btn btn-sm btn-secondary"),
                Button("Reset", type="button", cls="btn btn-sm btn-ghost",
                       onclick="this.form.sql.value=''; document.getElementById('sql-results').innerHTML='';"))),
        Div(id="sql-results", cls="mt-4")),
 

In [ ]:
#| export
def DbStats(cfg):
    path = Path(cfg.db_path)
    size_mb = path.stat().st_size / (1024 * 1024)
    return Div(cls="card bg-base-200 p-4 mb-6")(
        Div(cls="flex gap-8")(
            Div(Span("Database: ", cls="font-bold"), Span(path.name)),
            Div(Span("Size: ", cls="font-bold"), Span(f"{size_mb:.2f} MB")),
        )
    )

In [ ]:
p(DbStats(cfg))

In [ ]:
#| export
def TableStats(db, tbl):
    t = db.t[tbl]
    cols = get_cols(db, t)
    row_count = db.execute(f"SELECT COUNT(*) FROM \"{tbl}\"").fetchone()[0]
    col_types = get_col_types(db, t)
    
    return Div(cls="card bg-base-200 p-4 mb-4")(
        Div(cls="flex gap-8")(
            Div(Span("Rows: ", cls="font-bold"), Span(f"{row_count:,}")),
            Div(Span("Columns: ", cls="font-bold"), Span(str(len(cols)))),
        ),
        Div(cls="text-sm text-gray-500 mt-2")(
            ", ".join(f"{c} ({col_types[c]})" for c in cols)
        )
    )

In [ ]:
#| export
def LoginPage(*, req=_mock_req):
    return Div(cls="min-h-screen flex items-center justify-center")(
        Card(cls="w-full max-w-sm shadow-xl")(
            CardBody(
                H2("Welcome Back", cls="card-title justify-center mb-4"),
                Form(method="post", action=url(req, "/login"))(
                    LabeledInput("lock-closed", type="password", name="password", placeholder="Password", required=True),
                    Button("Login", cls="btn btn-primary w-full mt-4")))))

In [ ]:
p(LoginPage())

In [ ]:
#| export
def TableCard(name, row_count, col_count, *, req=_mock_req):
    return A(href=url(req, f"/tables/{name}"))(
        Card(cls="bg-base-200 hover:bg-base-300 transition-colors cursor-pointer")(
            CardBody(
                H3(name, cls="card-title"),
                Div(cls="flex gap-4 text-sm opacity-70")(
                    Span(f"{row_count:,} rows"),
                    Span(f"{col_count} columns")
                )
            )
        )
    )

In [ ]:
p(TableCard("Foobar", 123,123))

In [ ]:
#| export
def PageBtns(tbl, page, total_pages, q="", *, req=_mock_req):
    if total_pages <= 7:
        pages = list(range(1, total_pages + 1))
    else:
        pages = [1, 2]
        if page > 4: pages.append("...")
        pages.extend(i for i in range(max(3, page-1), min(total_pages-1, page+2)))
        if page < total_pages - 3: pages.append("...")
        pages.extend([total_pages-1, total_pages])
        pages = list(dict.fromkeys(pages))  # remove duplicates, preserve order
    return [Span("...", cls="px-2") if p == "..." else A(str(p), href=url(req, f"/tables/{tbl}?page={p}&q={q}"), cls=f"btn btn-sm {'btn-active' if p == page else ''}") for p in pages]

In [ ]:
#| export
def SearchForm(tbl, q="", *, req=_mock_req):
    return Form(cls="flex gap-2 items-center")(
        Input_(name="q", value=q, placeholder="Search...", cls="input input-bordered input-sm w-64",
               hx_get=url(req, f"/tables/{tbl}"), hx_trigger="input changed delay:300ms", hx_target="#table-content", hx_swap="innerHTML", hx_push_url="true"),
        A("Reset", href=url(req, f"/tables/{tbl}"), cls="btn btn-sm btn-ghost"))

## Authentication

In [ ]:
#| export
@rt("/login")
def get(req): return Layout(LoginPage(req=req), title="Admin Login", req=req, logged_in=False)

@rt("/login")
def post(req, password: str, sess):
    if password != app.state.cfg.password: return Layout(LoginPage(req=req), Alert("Invalid password", cls="alert-error mt-4"), title="Admin Login", req=req, logged_in=False)
    sess['auth'] = True
    return Redirect(url(req, "/tables"))

In [ ]:
#| export
@rt("/logout")
def post(req, sess):
    sess.pop('auth', None)
    return Redirect(url(req, "/login"))

In [ ]:
#| export
def auth_check(req, sess):
    if not sess.get('auth'): return Redirect(url(req, "/login"))

## Tables Overview

In [ ]:
#| export
def _stats(t):
    table, _, stats = t
    return table, int(stats.split()[0])

We need a fall back for when the `sqlite_stat1` table hasn't been populated yet, e.g. because noone has run `ANALYZE` yet. That's what the `try except` block does. If the first query fails, then we can fall back to looping over each table, which isn't as elegant but works.

In [ ]:
#| export
@rt("/tables")
def get(req, sess):
    if redir := auth_check(req, sess): return redir
    db = app.state.cfg.db
    try:
        row_counts = dict(L(db.execute("SELECT * FROM sqlite_stat1").fetchall()).map(_stats))
    except:
        row_counts = {tbl_name(t): db.execute(f'SELECT COUNT(*) FROM "{tbl_name(t)}"').fetchone()[0] for t in db.t}
    cards = [TableCard(tbl_name(t), row_counts.get(tbl_name(t), 0), len(get_cols(db, t)), req=req) for t in db.t]
    return Layout(
        DbStats(app.state.cfg),
        H1("Tables", cls="text-2xl font-bold mb-6"),
        Div(cls="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-4")(*cards),
        title="Tables", req=req, logged_in=True
    )

#| export
## Table Detail View

In [ ]:
#| export
@rt("/tables/{tbl}/{pk}/edit/{col}")
def get(req, sess, tbl: str, pk: str, col: str):
    if redir := auth_check(req, sess): return redir
    db = app.state.cfg.db
    row = db.t[tbl][pk]
    return EditCellInput(tbl, pk, col, row.get(col, ''), req=req)

In [ ]:
#| export
@rt("/tables/{tbl}/new")
def get(req, sess, tbl: str):
    if redir := auth_check(req, sess): return redir
    db = app.state.cfg.db
    return NewRowModal(tbl, get_cols(db, db.t[tbl]), req=req)

In [ ]:
#| export
@rt("/tables/{tbl}/new")
async def post(req, sess, tbl: str):
    if redir := auth_check(req, sess): return redir
    db = app.state.cfg.db
    form = await req.form()
    data = {k: v for k, v in form.items() if v}
    db.t[tbl].insert(**data)
    return Redirect(url(req, f"/tables/{tbl}"))

@rt("/tables/{tbl}/{pk}/edit/{col}")
async def put(req, sess, tbl: str, pk: str, col: str):
    if redir := auth_check(req, sess): return redir
    db = app.state.cfg.db
    form = await req.form()
    val = form.get(col, '')
    t = db.t[tbl]
    pk_col = get_cols(db, t)[0]
    db.execute(f"UPDATE {tbl} SET {col} = ? WHERE {pk_col} = ?", [val, pk])
    return EditableCell(tbl, pk, col, val, req=req)

delete row route

In [ ]:
#| export
@rt("/tables/{tbl}/{pk}")
def delete(req, sess, tbl: str, pk: str):
    if not sess.get('auth'): return Redirect(url(req, "/login"))
    db = app.state.cfg.db
    t = db.t[tbl]
    t.delete(pk)
    return ""

That's the route to make a sql query to the db.

In [ ]:
#| export
@rt("/tables/{tbl}/sql")
async def post(req, sess, tbl: str):
    if redir := auth_check(req, sess): return redir
    db = app.state.cfg.db
    form = await req.form()
    sql = form.get('sql', '').strip()
    if not sql: return Div("No query provided", cls="text-error")
    try:
        result = db.execute(sql).fetchall()
        if not result: return Div("Query executed successfully. No results.", cls="text-success")
        cols = [desc[0] for desc in db.execute(sql).description]
        header = Tr(*[Th(c) for c in cols])
        body = [Tr(*[Td(str(v)) for v in row]) for row in result[:100]]
        return Div(cls="overflow-x-auto")(Table(cls="table table-zebra table-sm")(Thead(header), Tbody(*body)),
            P(f"Showing {len(body)} of {len(result)} rows", cls="text-sm text-gray-500 mt-2") if len(result) > 100 else None)
    except Exception as e: return Div(f"Error: {e}", cls="text-error")

In [ ]:
#| export
def search_rows(db, tbl, cols, text_cols, q):
    if not (q and text_cols): return db.t[tbl]()
    where = " OR ".join([f"{c} LIKE ?" for c in text_cols])
    rows = db.execute(f"SELECT * FROM {tbl} WHERE {where}", [f"%{q}%" for _ in text_cols]).fetchall()
    return [dict(zip(cols, r)) for r in rows]

In [ ]:
#| export
def paginate(rows, page, per_page=25):
    total = len(rows)
    total_pages = max(1, (total + per_page - 1) // per_page)
    return rows[(page-1)*per_page : page*per_page], total_pages

In [ ]:
#| export
@rt("/tables/{tbl}")
def get(req, sess, tbl: str, page: int = 1, q: str = ""):
    if redir := auth_check(req, sess): return redir
    db = app.state.cfg.db
    t = db.t[tbl]
    cols = get_cols(db, t)
    all_rows = search_rows(db, tbl, cols, get_text_cols(db, t), q)
    rows, total_pages = paginate(all_rows, page)
    
    header = Tr(*[Th(c) for c in cols], Th("Actions"))
    
    table_content = Div(
        Table(cls="table table-zebra")(Thead(header), Tbody(*TableRows(rows, cols, tbl, req=req))),
        Div(cls="flex gap-2 mt-4 justify-center")(*PageBtns(tbl, page, total_pages, q, req=req)))
    
    # If HTMX request, return just the table content
    if req.headers.get("hx-request"):
        return table_content

    left = Div(cls="lg:col-span-2")(Div(cls="overflow-x-auto", id="table-content")(table_content))

    right = Div(cls="lg:col-span-1 flex flex-col gap-4")(
        Div(cls="card bg-base-200 p-4")(H3("Search", cls="font-bold mb-2"), SearchForm(tbl, q, req=req)),
        Div(cls="card bg-base-200 p-4")(SqlConsole(tbl, req=req)))

    return Layout(
        Div(cls="flex items-center justify-between mb-4")(
            A(Heroicon("arrow-left", cls="size-4"), "Back", href=url(req, "/tables"), cls="btn btn-ghost btn-sm gap-1"), H1(tbl, cls="text-2xl font-bold inline"),
            NewRowBtn(tbl, req=req)),
        TableStats(db, tbl),
        Div(cls="grid grid-cols-1 lg:grid-cols-3 gap-6")(left, right),
        title=tbl, req=req, logged_in=True)

## Server

In [ ]:
if 'srv' in globals():
    srv.stop()

In [ ]:
#|eval: false
srv = JupyUvi(app)

### Test mounting

In [ ]:
parent = FastHTML()

@parent.get("/foobar")
def home(): return H1("Parent App"), A("Go to Admin", href="/admin/tables")

parent.mount("/admin", app)

In [ ]:
#|eval: false
if "srv" in globals(): srv.stop()
srv = JupyUvi(parent)

In [ ]:
app.routes

In [ ]:
parent.routes

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()